# MovieLens users and movies embaddings

In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 162.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 4.5 MB/s eta 0:00:0000:0100:010m
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.9 MB/s eta 0:00:0000:01:00:01
  Using cached scikit_learn-1.2.2-cp39-cp39-macosx_12_0_arm64.whl (8.5 MB)
  Using cached scipy-1.10.1-cp39-cp39-macosx_12_0_arm64.whl (28.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.2 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.

In [ ]:
import urllib.request
import io
import zipfile
import pandas as pd
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

## Data load and preparation

Создаем ссылку на датасет и словарь, в котором будут храниться таблицы

In [4]:
url = 'https://files.grouplens.org/datasets/movielens/ml-latest.zip'
datasets = {}

zipfile.namelist() используется для получения списка имен файлов в архиве, затем отбираются только те файлы, которые заканчиваются на '.csv'. Для каждого .csv файла используется pandas.read_csv() для загрузки данных в датафрейм, затем этот датафрейм добавляется в словарь datasets

In [5]:
with urllib.request.urlopen(url) as url_file:
  with zipfile.ZipFile(io.BytesIO(url_file.read())) as zip_file:
    for file_name in zip_file.namelist():
      if file_name.endswith('.csv'):
        with zip_file.open(file_name) as data_file:
          dataset = pd.read_csv(data_file)
          datasets[file_name] = dataset

Проверим

In [6]:
datasets.keys()

dict_keys(['ml-latest/links.csv', 'ml-latest/tags.csv', 'ml-latest/genome-tags.csv', 'ml-latest/ratings.csv', 'ml-latest/genome-scores.csv', 'ml-latest/movies.csv'])

Загружаем все датафреймы в отельные переменные для удобства

In [7]:
links = datasets['ml-latest/links.csv']
tags = datasets['ml-latest/tags.csv']
genome_tags = datasets['ml-latest/genome-tags.csv']
ratings = datasets['ml-latest/ratings.csv']
genome_scores = datasets['ml-latest/genome-scores.csv']
movies = datasets['ml-latest/movies.csv']

Так как нас интересуют пользователи и фильмы, объединяем таблицы с помощью айдишника фильма

In [8]:
data = pd.merge(ratings, movies, on='movieId')
data

## Keras solution

Для построения нейронки воспользуемся моделью "Embedding", которая позволит нам получить векторное представление для каждого пользователя и фильма. Это достигается путем использования слоя "Embedding" в модели, который принимает номер пользователя или фильма и возвращает векторное представление для этих данных

Количество уникальных пользователей и фильмов

In [9]:
n_users = len(data.userId.unique())
n_movies = len(data.movieId.unique())

Входные данные для пользователей

In [39]:
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=n_users, output_dim=50, name='user_embedding')(user_input)
user_flatten = Flatten(name='user_flatten')(user_embedding)

Входные данные для фильмов

In [40]:
movie_input = Input(shape=(1,), name='movie_input')
movie_embedding = Embedding(input_dim=n_movies, output_dim=50, name='movie_embedding')(movie_input)
movie_flatten = Flatten(name='movie_flatten')(movie_embedding)

Объединяем векторы пользователей и фильмов

In [41]:
dot = Dot(name='dot', axes=1)([user_flatten, movie_flatten])

Добавляем скрытый слой

In [43]:
dense = Dense(128, activation='relu')(dot)

Добавляем выходной слой

In [44]:
output = Dense(1, activation='linear')(dense)

Создаем модель

Воспользуемся метрикой RMSE, которая является стандартной метрикой для задач регрессии. Эта метрика показывает насколько отличаются предсказанные значения от фактических. В нашем случае, метрика покажет насколько отличается рейтинг, предсказанный моделью, от реального рейтинга

In [45]:
model = Model(inputs=[user_input, movie_input], outputs=output)
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 movie_input (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 user_embedding (Embedding)     (None, 1, 50)        14161400    ['user_input[0][0]']             
                                                                                                  
 movie_embedding (Embedding)    (None, 1, 50)        2694450     ['movie_input[0][0]']            
                                                                                            

Стандартно разделяем на трейн/тест

In [46]:
train, test = train_test_split(data, test_size=0.2)

Обучаем модель

In [47]:
history = model.fit(
    [train.userId, train.movieId],
    train.rating,
    batch_size=64,
    epochs=5,
    validation_split=0.1,
    verbose=1
)

Epoch 1/5
312227/312227 [==============================] - 2358s 7ms/step - loss: 1.0402 - val_loss: 1.0094
Epoch 2/5
312227/312227 [==============================] - 2329s 7ms/step - loss: 0.9627 - val_loss: 0.9822
Epoch 3/5
312227/312227 [==============================] - 2324s 7ms/step - loss: 0.9085 - val_loss: 0.9683
Epoch 4/5
312227/312227 [==============================] - 2308s 7ms/step - loss: 0.8523 - val_loss: 0.9653
Epoch 5/5
312227/312227 [==============================] - 2297s 7ms/step - loss: 0.8209 - val_loss: 0.9675


In [48]:
loss = model.evaluate([test.userId, test.movieId], test.rating)

173460/173460 [==============================] - 367s 2ms/step - loss: 0.9666


Получаем эмбеддинги

In [49]:
user_embeddings = model.get_layer('user_embedding').get_weights()[0]
movie_embeddings = model.get_layer('movie_embedding').get_weights()[0]

Теперь мы можем получить эмбеддинг любого пользователя или фильма для дальнейшего построения рекомендательной системы. Примером такой системы могут послужить методы совместной фильтрации или фильтрации на основе содержимого

In [51]:
user_embeddings[0]

array([-0.02779937,  0.00534084,  0.04119035,  0.0403241 ,  0.04175929,
        0.00733225,  0.02360013,  0.04125011,  0.01223646, -0.04585896,
       -0.03797797,  0.04649505,  0.01777902,  0.02611933, -0.03935809,
        0.00966378,  0.02512116,  0.00511296, -0.04630522,  0.03667393,
        0.0027166 , -0.04555022, -0.01216433,  0.00612092,  0.04418515,
        0.03996499, -0.02412491, -0.03723481, -0.00687162,  0.01643166,
       -0.04190822, -0.02582397,  0.00117594, -0.02756792, -0.0120428 ,
       -0.02693969, -0.00619071, -0.00637976, -0.04316893,  0.01887869,
       -0.01709574,  0.00214823, -0.04281486, -0.01752869,  0.04286965,
       -0.03691769, -0.04671529,  0.01090802, -0.00251915,  0.02390409],
      dtype=float32)

In [50]:
movie_embeddings[0]

array([-0.04293933,  0.03131613, -0.01073663, -0.03731848,  0.03326725,
        0.04103079,  0.00210664, -0.03034762,  0.01994571,  0.04412626,
        0.00429127,  0.03168922, -0.03301521, -0.03680787,  0.03625906,
        0.03576152,  0.01582966, -0.02679781, -0.04903481, -0.00113448,
       -0.01390636, -0.04822518,  0.00497645, -0.02654135, -0.00971418,
       -0.0454193 , -0.01934587,  0.039131  , -0.04413257,  0.0487512 ,
       -0.03569707, -0.03424858,  0.01815159,  0.03038087, -0.021184  ,
       -0.036779  , -0.01747854, -0.00431689, -0.00383417,  0.02753878,
        0.02891609,  0.00415385,  0.03766796,  0.02432663, -0.01944426,
        0.01731909, -0.02475315, -0.04089544, -0.01920109,  0.01414583],
      dtype=float32)

## SentenceTransformer solution

Другой метод решения – использование Bert Universal Sentence Encoder для получения эмбеддинга тайтлов фильмов. После мы можем посчитать средний эмбединг всех просмотренных пользователем фильмов с учетом их рейтинга и присвоить их ему. Дальше на основе этого можно рекомендовать фильмы с похожими эмбеддингами

Получаем уникальные названия фильмов

In [10]:
unique_titles = data['title'].unique()
unique_titles

array(['Three Colors: Blue (Trois couleurs: Bleu) (1993)',
       'Kalifornia (1993)', "Weekend at Bernie's (1989)", ...,
       'Hotline (2014)', 'Barnum! (1986)',
       'Paul Taylor Creative Domain (2014)'], dtype=object)

Создаем словарь с парами названия фильма: эмбеддинг

In [23]:
title_encodings = {}

Создаем трансформер

In [12]:
model = SentenceTransformer('sentence-transformers/use-cmlm-multilingual')

Some weights of the model checkpoint at /Users/alekseykomissarenko/.cache/torch/sentence_transformers/sentence-transformers_use-cmlm-multilingual/ were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Далее получаем эмбеддинги для каждого уникального названия фильма

In [ ]:
for i in tqdm(range(len(unique_titles))):
  encoding = model.encode(unique_titles[i])
  title_encodings[unique_titles[i]] = encoding

  0%|          | 0/53817 [00:00<?, ?it/s]

In [14]:
for i in tqdm(range(37439, len(unique_titles))):
    encoding = model.encode(unique_titles[i])
    title_encodings[unique_titles[i]] = encoding

  0%|          | 0/16378 [00:00<?, ?it/s]

In [17]:
len(title_encodings)

53817

Создаем колонку с эмбедингами

In [21]:
encodings_column = []

Записываем эмбеддинг, если название фильма в строке совпадает с ключом словаря эмбеддингов

In [22]:
for row in tqdm(range(len(data))):
    for encoding in title_encodings:
        if encoding == data['title'][row]:
            encodings_column.append(encoding)

  0%|          | 0/27753444 [00:00<?, ?it/s]

KeyboardInterrupt: 

К сожалению, не успел по времени, поэтому дальнейшее решение по плану продолжить не удалось